In [ ]:
!pip install lightgbm

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/


In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, mean_squared_error
import tqdm
import sys
import os
import gc
import argparse
import warnings
warnings.filterwarnings('ignore')


def time_model(lgb, train_df, test_df, cols):
 

    # 训练集和验证集切分
    trn_x, trn_y = train_df[train_df.dt>=31][cols], train_df[train_df.dt>=31]['target']
    val_x, val_y = train_df[train_df.dt<=30][cols], train_df[train_df.dt<=30]['target']
    # 构建模型输入数据
    train_matrix = lgb.Dataset(trn_x, label=trn_y)
    valid_matrix = lgb.Dataset(val_x, label=val_y)
    # lightgbm参数
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'mse',
        'min_child_weight': 5,
        'num_leaves': 2 ** 5,
        'lambda_l2': 10,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 4,
        'learning_rate': 0.025,
        'seed': 2024,
        'nthread' : 16,
        'verbose' : -1,
        'early_stopping_round' : 500
        # 'verbose_eval' : 500,
    }

    # 训练模型
    model = lgb.train(lgb_params, train_matrix, 50000, valid_sets=[train_matrix, valid_matrix], 
                      categorical_feature=[], callbacks=[print_evaluation(period=500)])
    
    # 打印评估回调函数
    def print_evaluation(period=1):
        def callback(env):
            if (env.iteration + 1) % period == 0:
                print(f'Iteration {env.iteration + 1}, evaluation result: {env.evaluation_result_list}')
        return callback

    # 验证集和测试集结果预测
    val_pred = model.predict(val_x, num_iteration=model.best_iteration)
    test_pred = model.predict(test_df[cols], num_iteration=model.best_iteration)
       
    # 离线分数评估
    score = mean_squared_error(val_pred, val_y)
    print(score)
    
    return val_pred, test_pred
    
lgb_oof, lgb_test = time_model(lgb, train, test, train_cols)

# 保存结果文件到本地
test['target'] = lgb_test
test[['id','dt','target']].to_csv('submit.csv', index=None)

Iteration 500, evaluation result: [('training', 'l2', 192.34290910999715, False), ('valid_1', 'l2', 189.92740126346237, False)]
Iteration 1000, evaluation result: [('training', 'l2', 181.36818819995142, False), ('valid_1', 'l2', 185.3841855788365, False)]
Iteration 1500, evaluation result: [('training', 'l2', 175.12588271488517, False), ('valid_1', 'l2', 184.22496899410484, False)]
Iteration 2000, evaluation result: [('training', 'l2', 170.5459611881833, False), ('valid_1', 'l2', 184.02657881024567, False)]
183.6860000177447
